In [1]:
from __future__ import annotations

import os
from typing import Iterable, List, Optional

import httpx
import polars as pl


FRED_BASE = "https://api.stlouisfed.org/fred"

In [2]:
def _chunked(xs: List[str], n: int) -> Iterable[List[str]]:
    for i in range(0, len(xs), n):
        yield xs[i : i + n]


def _request_with_retry(client: httpx.Client, url: str, params: dict,
                        timeout: float = 30.0, max_retries: int = 6) -> httpx.Response:
    """GET with exponential backoff on 429 and transient 5xx errors."""
    import time as _time
    for attempt in range(max_retries):
        r = client.get(url, params=params, timeout=timeout)
        if r.status_code == 429 or r.status_code >= 500:
            wait = min(2 ** attempt, 60)
            print(f"    [{r.status_code}] retrying in {wait}s ...")
            _time.sleep(wait)
            continue
        r.raise_for_status()
        return r
    r.raise_for_status()
    return r


def get_vintage_dates(
    client: httpx.Client,
    series_id: str,
    api_key: str,
) -> List[str]:
    r = _request_with_retry(
        client,
        f"{FRED_BASE}/series/vintagedates",
        params={
            "series_id": series_id,
            "api_key": api_key,
            "file_type": "json",
        },
        timeout=30.0,
    )
    data = r.json()
    return data.get("vintage_dates", [])


def get_observations_for_vintages(
    client: httpx.Client,
    series_id: str,
    api_key: str,
    vintage_dates: List[str],
    output_type: int = 2,
    chunk_size: int = 200,
    observation_start: Optional[str] = None,
) -> pl.DataFrame:
    """
    Pull observations for a set of vintage_dates.

    Notes:
    - FRED allows comma-separated vintage_dates; chunking avoids query-length issues.
    - output_type=2 returns wide-format with one column per vintage.
    """
    frames: List[pl.DataFrame] = []

    for chunk in _chunked(vintage_dates, chunk_size):
        params = {
            "series_id": series_id,
            "api_key": api_key,
            "file_type": "json",
            "vintage_dates": ",".join(chunk),
            "output_type": str(output_type),
        }
        if observation_start is not None:
            params["observation_start"] = observation_start
        r = _request_with_retry(
            client,
            f"{FRED_BASE}/series/observations",
            params=params,
            timeout=60.0,
        )
        obs = r.json().get("observations", [])
        if not obs:
            continue

        frames.append(pl.from_dicts(obs))

    if not frames:
        return pl.DataFrame(schema={"date": pl.Utf8})

    df = pl.concat(frames, how="vertical_relaxed")
    return df


def compute_initial_and_latest_levels(df_wide: pl.DataFrame) -> pl.DataFrame:
    """
    From ALFRED output_type=2 wide-format observations, compute per date:
      - initial_level: value from the earliest vintage
      - latest_level:  value from the most recent vintage

    Wide format has columns: date, {SERIES}_{YYYYMMDD}, ...
    Vintage columns are ordered chronologically left-to-right.
    """
    vintage_cols = [c for c in df_wide.columns if c != "date"]
    if not vintage_cols:
        return pl.DataFrame(schema={"date": pl.Date, "initial_level": pl.Float64, "latest_level": pl.Float64})

    def _to_float(col_name: str) -> pl.Expr:
        return (
            pl.col(col_name)
            .cast(pl.Utf8)
            .str.replace_all(r"^\.$", "")
            .str.strip_chars()
            .cast(pl.Float64, strict=False)
        )

    df = df_wide.with_columns(
        pl.col("date").str.to_date(strict=False).alias("date"),
        *[_to_float(c).alias(c) for c in vintage_cols],
    ).filter(pl.col("date").is_not_null())

    df = df.with_columns(
        pl.coalesce([pl.col(c) for c in vintage_cols]).alias("initial_level"),
        pl.coalesce([pl.col(c) for c in reversed(vintage_cols)]).alias("latest_level"),
    ).select("date", "initial_level", "latest_level").sort("date")

    return df


def fetch_levels_initial_vs_latest(
    series_id: str,
    fred_api_key: Optional[str] = None,
    last_n_vintages: Optional[int] = 24,
    chunk_size: int = 200,
    observation_start: Optional[str] = None,
) -> pl.DataFrame:
    api_key = fred_api_key or os.environ["FRED_API_KEY"]

    with httpx.Client() as client:
        vintage_dates = get_vintage_dates(client, series_id=series_id, api_key=api_key)

        if last_n_vintages is not None:
            vintage_dates = vintage_dates[-last_n_vintages:]

        df_obs = get_observations_for_vintages(
            client,
            series_id=series_id,
            api_key=api_key,
            vintage_dates=vintage_dates,
            output_type=2,
            chunk_size=chunk_size,
            observation_start=observation_start,
        )

    return compute_initial_and_latest_levels(df_obs)

In [3]:
FIPS_TO_ABBREV = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
    '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
    '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
    '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
    '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
    '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
    '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
    '56': 'WY', '72': 'PR', '78': 'VI',
}

# (ces_8digit_code, name, level, short_suffix_or_None)
#
# short_suffix: used for {ABBREV}{SUFFIX} IDs (~222 vintages from 2007)
# None:         falls back to SMU{FIPS}00000{CES_8}01 (~139 vintages from 2014)
INDUSTRIES = [
    
    # Domains
    ('00000000', '00', 'Total Nonfarm', 'domain', 'NAN'),
    ('05000000', '05', 'Total Private', 'domain', None),
    ('06000000', '06', 'Goods-Producing', 'domain', None),
    ('07000000', '07', 'Service-Providing', 'domain', None),
    ('08000000', '08', 'Private Service-Providing', 'domain', None),
    
    # Supersectors
    ('10000000', '10', 'Natural Resources and Mining', 'supersector', 'NRMNN'),
    ('20000000', '20', 'Construction', 'supersector', 'CONSN'),
    ('30000000', '30', 'Manufacturing', 'supersector', 'MFGN'),
    ('40000000', '40', 'Trade, Transportation, and Utilities', 'supersector', 'TRADN'),
    ('50000000', '50', 'Information', 'supersector', 'INFON'),
    ('55000000', '55', 'Financial Activities', 'supersector', 'FIREN'),
    ('60000000', '60', 'Professional and Business Services', 'supersector', 'PBSVN'),
    ('65000000', '65', 'Education and Health Services', 'supersector', 'EDUHN'),
    ('70000000', '70', 'Leisure and Hospitality', 'supersector', 'LEIHN'),
    ('80000000', '80', 'Other Services', 'supersector', 'SRVON'),
    ('90000000', '90', 'Government', 'supersector', 'GOVTN'),

    # Sectors (SMU long-form only; some may not exist for every state)
    ('10210000', '21', 'Mining', 'sector', None),
    ('31000000', '31', 'Durable Goods', 'sector', None),
    ('32000000', '32', 'Nondurable Goods', 'sector', None),
    ('41000000', '41', 'Wholesale Trade', 'sector', None),
    ('42000000', '42', 'Retail Trade', 'sector', None),
    ('43000000', '43', 'Transp., Warehousing & Utilities', 'sector', None),
    ('43220000', '22', 'Utilities', 'sector', None),
    ('55520000', '52', 'Finance and Insurance', 'sector', None),
    ('55530000', '53', 'Real Estate', 'sector', None),
    ('60540000', '54', 'Prof., Scientific & Tech. Services', 'sector', None),
    ('60550000', '55', 'Management of Companies', 'sector', None),
    ('60560000', '56', 'Admin. and Support Services', 'sector', None),
    ('65610000', '61', 'Private Educational Services', 'sector', None),
    ('65620000', '62', 'Health Care and Social Assistance', 'sector', None),
    ('70710000', '71', 'Arts, Entertainment & Recreation', 'sector', None),
    ('70720000', '72', 'Accommodation and Food Services', 'sector', None),
    ('90910000', '91', 'Federal Government', 'sector', None),
    ('90920000', '92', 'State Government', 'sector', None),
    ('90930000', '93', 'Local Government', 'sector', None),
]

def _make_series_id(fips: str, abbrev: str, ces_code: str,
                    nsa_suffix: Optional[str], adjusted: bool) -> str:
    if nsa_suffix is not None:
        suffix = nsa_suffix[:-1] if adjusted else nsa_suffix
        return f"{abbrev}{suffix}"
    prefix = "SMS" if adjusted else "SMU"
    return f"{prefix}{fips}00000{ces_code}01"


rows = []
for a in ['SA', 'NSA']:
    adjusted = True if a == 'SA' else False
    for fips, abbrev in FIPS_TO_ABBREV.items():
        for ces_code, code, name, level, nsa_suffix in INDUSTRIES:
            if code == '00':
                rows.append({   
                    'series_id': _make_series_id(fips, abbrev, ces_code, nsa_suffix, adjusted),
                    'adjusted': adjusted,
                    'geographic_type': 'state',
                    'geographic_code': fips,
                    'state_fips': fips,
                    'state_abbrev': abbrev,
                    'ces_industry': ces_code,
                    'industry_type': level,
                    'industry_code': code,
                    'industry_name': name,
                })

series_df = pl.DataFrame(rows)
print(
    f"{len(series_df)} series |  "
    f"{series_df['state_fips'].n_unique()} states  |  "
    f"{series_df['ces_industry'].n_unique()} industries"
)

106 series |  53 states  |  1 industries


In [6]:
def fetch_batch_revisions(
    series_df: pl.DataFrame,
    fred_api_key: str,
    last_n_vintages: int = 24,
    chunk_size: int = 200,
    sleep_between: float = 1.0,
    observation_start: Optional[str] = None,
) -> pl.DataFrame:
    """Fetch initial-vs-latest levels for every series in *series_df*.

    Returns a long DataFrame with columns:
        date, initial_level, latest_level,
        series_id, state_fips, state_abbrev, ces_industry, industry_name
    Series that don't exist on FRED (HTTP 400) are silently skipped.
    """
    api_key = fred_api_key
    results: list[pl.DataFrame] = []
    skipped: list[str] = []
    total = len(series_df)

    with httpx.Client() as client:
        for i, row in enumerate(series_df.iter_rows(named=True)):
            sid = row["series_id"]
            try:
                vdates = get_vintage_dates(client, series_id=sid, api_key=api_key)
                if last_n_vintages is not None:
                    vdates = vdates[-last_n_vintages:]
                if not vdates:
                    continue

                df_obs = get_observations_for_vintages(
                    client, series_id=sid, api_key=api_key,
                    vintage_dates=vdates, output_type=2, chunk_size=chunk_size,
                    observation_start=observation_start,
                )
                levels = compute_initial_and_latest_levels(df_obs)
                if levels.height > 0:
                    levels = levels.with_columns(
                        pl.lit(sid).alias("series_id"),
                        pl.lit(row["adjusted"]).alias("adjusted"),
                        pl.lit(row["state_fips"]).alias("state_fips"),
                        pl.lit(row["state_abbrev"]).alias("state_abbrev"),
                        pl.lit(row["ces_industry"]).alias("ces_industry"),
                        pl.lit(row["geographic_type"]).alias("geographic_type"),
                        pl.lit(row["geographic_code"]).alias("geographic_code"),
                        pl.lit(row["industry_name"]).alias("industry_name"),
                        pl.lit(row["industry_type"]).alias("industry_type"),
                        pl.lit(row["industry_code"]).alias("industry_code"),
                    )
                    results.append(levels)

                if (i + 1) % 25 == 0:
                    print(f"  [{i+1}/{total}] fetched {len(results)} series so far ({len(skipped)} skipped)")

            except httpx.HTTPStatusError as e:
                if e.response.status_code in (400, 404):
                    skipped.append(sid)
                else:
                    raise

            time.sleep(sleep_between)

    print(f"\nDone: {len(results)} series fetched, {len(skipped)} not found on FRED")
    if skipped:
        print(f"  Skipped (first 20): {skipped[:20]}")

    if not results:
        return pl.DataFrame()
        
    return (
        pl
        .concat(
            results, 
            how="vertical_relaxed"
        )
        .select(
            ref_date=pl.col('date')
                       .dt.offset_by('11d'),
            adjusted=pl.col('adjusted'),
            geographic_type=pl.col('geographic_type'),
            geographic_code=pl.col('geographic_code'),
            industry_type=pl.col('industry_type'),
            industry_code=pl.col('industry_code'),
            employment_initial=pl.col('initial_level'),
            employment_latest=pl.col('latest_level'),
        )
    )

In [7]:
FRED_API_KEY = "8d08f0f04f7d3e53fbdd765c0bbfb329"
OBS_START = "2016-01-01"

sae_revisions = fetch_batch_revisions(
    series_df, 
    fred_api_key=FRED_API_KEY, 
    observation_start=OBS_START,
)

  [25/106] fetched 25 series so far (0 skipped)
  [50/106] fetched 50 series so far (0 skipped)
  [75/106] fetched 73 series so far (2 skipped)
  [100/106] fetched 98 series so far (2 skipped)

Done: 102 series fetched, 4 not found on FRED
  Skipped (first 20): ['PRNA', 'VINA', 'PRNAN', 'VINAN']


In [8]:
sae_revisions_nsa = (
    sae_revisions
    .filter(
        pl.col('adjusted').eq(False)
    )
)

sae_revisions_nsa_0 = (
    sae_revisions_nsa
    .rename({
        'adjusted': 'seasonally_adjusted',
        'employment_initial': 'employment'
    })
    .drop('employment_latest')
    .with_columns(
        revision=pl.lit(0, pl.UInt8)
    )
    .filter(
        pl.col('employment').is_not_null()
    )
)

sae_revisions_nsa_1 = (
    sae_revisions_nsa
    .rename({
        'adjusted': 'seasonally_adjusted',
        'employment_latest': 'employment'
    })
    .drop('employment_initial')
    .with_columns(
        revision=pl.lit(1, pl.UInt8)
    )
    .filter(
        pl.col('employment').is_not_null()
    )
)

print(f'Number of SAE NSA revision 0 observations: {sae_revisions_nsa_0.height: ,}')
print(f'Number of SAE NSA revision 1 observations: {sae_revisions_nsa_1.height: ,}')

Number of SAE NSA revision 0 observations:  6,120
Number of SAE NSA revision 1 observations:  6,120


In [9]:
sae_revisions_sa = (
    sae_revisions
    .filter(
        pl.col('adjusted').eq(True)
    )
)

sae_revisions_sa_0 = (
    sae_revisions_sa
    .rename({
        'adjusted': 'seasonally_adjusted',
        'employment_initial': 'employment'
    })
    .drop('employment_latest')
    .with_columns(
        revision=pl.lit(0, pl.UInt8)
    )
    .filter(
        pl.col('employment').is_not_null()
    )
)

sae_revisions_sa_1 = (
    sae_revisions_sa
    .rename({
        'adjusted': 'seasonally_adjusted',
        'employment_latest': 'employment'
    })
    .drop('employment_initial')
    .with_columns(
        revision=pl.lit(1, pl.UInt8)
    )
    .filter(
        pl.col('employment').is_not_null()
    )
)

print(f'Number of SAE SA revision 0 observations: {sae_revisions_sa_0.height: ,}')
print(f'Number of SAE SA revision 1 observations: {sae_revisions_sa_1.height: ,}')

Number of SAE SA revision 0 observations:  6,120
Number of SAE SA revision 1 observations:  6,120


In [10]:
sae_revisions_1 = (
    pl
    .concat(
        [
            sae_revisions_nsa_0,
            sae_revisions_nsa_1,
            sae_revisions_sa_0,
            sae_revisions_sa_1,
        ]
    )
    .with_columns(
        source=pl.lit('sae', pl.Utf8)
    )
)

print(f'Number of SAE revision observations: {sae_revisions_1.height: ,}')

Number of SAE revision observations:  24,480


In [11]:
vintage_dates = (
    pl
    .read_parquet(
        '/Users/lowell/Projects/bls-release-dates/data/vintage_dates.parquet'
    )
    .filter(
        pl.col('publication').eq('sae')
    )
    .select(
        ref_date=pl.col('ref_date'),
        revision=pl.col('revision')
                   .cast(pl.UInt8),
        benchmark_revision=pl.col('benchmark_revision')
                             .cast(pl.UInt8),
        vintage_date=pl.col('vintage_date')
    )
)

In [12]:
sae_revisions_df = (
    sae_revisions_1
    .join(
        vintage_dates,
        on=['ref_date', 'revision'],
        how='left'
    )
    .select(
        'source',
        'seasonally_adjusted',
        'geographic_type', 'geographic_code', 
        'industry_type', 'industry_code', 
        'ref_date', 'vintage_date',
        'revision', 'benchmark_revision', 
        'employment'
    )
    .filter(
        pl.col('vintage_date').is_not_null()
    )
)

print(f'Number of SAE revision observations (w/ dates): {sae_revisions_df.height: ,}')

#assert sae_revisions_df.height == sae_revisions_1.height

Number of SAE revision observations (w/ dates):  24,174


In [13]:
sae_revisions_dups = (
    sae_revisions_df
    .unique(
        subset=[
            'source',
            'seasonally_adjusted',
            'geographic_type', 'geographic_code', 
            'industry_type', 'industry_code', 
            'ref_date', 'vintage_date',
            'revision', 'benchmark_revision', 
        ]
    )
)

assert sae_revisions_df.height == sae_revisions_dups.height

In [15]:
(
    sae_revisions_df
    .write_parquet(
        '/Users/lowell/Projects/bls-revisions/data/sae_revisions.parquet'
    )
)